# **Evaluacion y analisis de modelos, casos porblema de regresión y clasificación**

### **Librerias:**

In [ ]:
# Importing necessary libraries
import pandas as pd  # For data manipulation and analysis
import matplotlib.pyplot as plt  # For data visualization
import time  # For tracking execution time

# Importing modules from scikit-learn for model building and evaluation
from sklearn.model_selection import train_test_split, GridSearchCV  # For splitting data and hyperparameter tuning
from sklearn.pipeline import Pipeline  # For creating machine learning pipelines
from sklearn.preprocessing import StandardScaler  # For feature scaling
from sklearn.linear_model import Ridge, Lasso  # For regression models
from sklearn.ensemble import RandomForestRegressor  # For ensemble regression model

# Importing XGBoost library for regression
from xgboost import XGBRegressor  # For gradient boosting regression

# Importing metrics for model evaluation
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error  # For evaluating regression models

In [5]:
df = pd.read_csv('..\df_final.csv')  # Load the dataset


## **Caso problema: Evaluación del potencial de energía Solar en Barranquilla: Análisis de irradiancia y variables atmosféricas para la generación sostenible**

In [6]:
# === Definición de variables predictoras y objetivo para el modelo de regresión ===

# Lista de variables independientes relevantes para predicción solar
feature_cols = [
    'T2M',       # Temperatura a 2 metros
    'RH2M',      # Humedad relativa a 2 metros
    'WS10M',     # Velocidad del viento a 10 metros
    'WD10M',     # Dirección del viento a 10 metros
    'PS',        # Presión superficial
    'T2MDEW',    # Punto de rocío a 2 metros
    'T2MWET',    # Temperatura de bulbo húmedo
    'WS50M'      # Velocidad del viento a 50 metros
]

# Variables predictoras (X) y variable objetivo (y) para regresión
X = df[feature_cols]
y = df['SolarIndex']



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_results = {}

### **Ridge Regression**

In [ ]:
ridge_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Ridge())
])

ridge_params = {'model__alpha': [0.01, 0.1, 1, 10, 100]}
ridge_grid = GridSearchCV(ridge_pipe, ridge_params, cv=5)

start_time = time.time()
ridge_grid.fit(X_train, y_train)
end_time = time.time()

y_pred_ridge = ridge_grid.predict(X_test)

ridge_training_time = end_time - start_time

model_results['Ridge'] = {
    'Best Alpha': ridge_grid.best_params_['model__alpha'],
    'R2': r2_score(y_test, y_pred_ridge),
    'MSE': mean_squared_error(y_test, y_pred_ridge),
    'MAE': mean_absolute_error(y_test, y_pred_ridge),
    'Training Time (s)': round(ridge_training_time, 2)
}



### **Lasso Regression**

In [ ]:
lasso_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Lasso(max_iter=10000))
])

lasso_params = {'model__alpha': [0.01, 0.1, 1, 10]}
lasso_grid = GridSearchCV(lasso_pipe, lasso_params, cv=5)
lasso_grid.fit(X_train, y_train)
y_pred_lasso = lasso_grid.predict(X_test)

model_results['Lasso'] = {
    'Best Alpha': lasso_grid.best_params_['model__alpha'],
    'R2': r2_score(y_test, y_pred_lasso),
    'MSE': mean_squared_error(y_test, y_pred_lasso),
    'MAE': mean_absolute_error(y_test, y_pred_lasso)
}



### **Random Forest**

In [ ]:
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

model_results['Random Forest'] = {
    'R2': r2_score(y_test, y_pred_rf),
    'MSE': mean_squared_error(y_test, y_pred_rf),
    'MAE': mean_absolute_error(y_test, y_pred_rf)
}

### **XGBoost**

In [ ]:
xgb = XGBRegressor(n_estimators=100, random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

model_results['XGBoost'] = {
    'R2': r2_score(y_test, y_pred_xgb),
    'MSE': mean_squared_error(y_test, y_pred_xgb),
    'MAE': mean_absolute_error(y_test, y_pred_xgb)
}

### **Resultados:**

#### **Comparacion grafica:**

In [3]:


# Convertir a DataFrame si no lo tienes
result_df_reg = pd.DataFrame(model_results).T

# Crear gráfico de barras para cada métrica
metrics = ['R2', 'MSE', 'MAE']

for metric in metrics:
    plt.figure(figsize=(8, 5))
    result_df_reg[metric].plot(kind='bar', color='skyblue')
    plt.title(f'Comparación de modelos según {metric}')
    plt.ylabel(metric)
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


NameError: name 'model_results' is not defined

## **Caso problema: Clasificación de islas de calor urbanas en Barranquilla: Análisis de variables meteorológicas para la caracterización del microclima local**

## **Balanceo de clases**

En *Machine Learning*, una clase se refiere a una de las categorías en las que se agrupan las observaciones dentro de un problema de clasificación. En un conjunto de datos etiquetado, cada instancia está asociada a una clase específica, permitiendo que un modelo aprenda a predecir la categoría correcta para nuevas observaciones.
Cuando el número de ejemplos en cada clase está desproporcionado, se habla de un **desbalance de clases**. Esto ocurre cuando una clase tiene significativamente más ejemplos que otra(s), afectando el desempeño del modelo de clasificación. En estas situaciones, el algoritmo tiende a favorecer la clase mayoritaria, lo que puede llevar a una alta precisión global, pero con un desempeño deficiente en la clase minoritaria.

Para mitigar este problema, se emplean técnicas de balanceo de clases, cuyo objetivo es modificar la distribución de los datos o ajustar la forma en que el modelo aprende. En este estudio, se implementarán tres enfoques principales:

* SMOTE (Synthetic Minority Over-sampling Technique): Es una técnica de sobremuestreo que genera nuevas instancias sintéticas de la clase minoritaria a partir de interpolaciones entre ejemplos existentes.
* ADASYN (Adaptive Synthetic Sampling): Similar a SMOTE, pero con la particularidad de generar más ejemplos en aquellas regiones donde la clase minoritaria es más dispersa, mejorando la representatividad del conjunto de datos.
* Ajuste de pesos con `class_weight='balanced`: Método que modifica la función de costo del modelo para penalizar más los errores en la clase minoritaria, sin necesidad de generar nuevos datos.


## **Contextualización**

El objetivo principal de este estudio es analizar cómo las condiciones meteorológicas varían en distintas áreas geográficas dentro de la misma ciudad. Para ello, se ha considerado como variable objetivo la "localidad". Esta variable se seleccionó debido a que las localidades en la ciudad pueden presentar diferencias significativas en las condiciones climáticas debido a factores geográficos, como la proximidad a cuerpos de agua, la altitud o la urbanización. Las diferencias en la temperatura, la precipitación, la humedad y otros factores climáticos pueden ser significativas entre localidades, lo que lleva a la formación de microclimas.

La localidad se define como una zona geográfica que agrupa uno o más barrios, lo que facilita el análisis de variabilidad no solo a nivel de ciudad, sino también de pequeñas áreas urbanas dentro de la misma. Este enfoque permite identificar patrones de variabilidad climática que podrían ser esenciales para la planificación urbana, el manejo de recursos naturales o la toma de decisiones a nivel local.

### **Verificación de balance de clases**

In [ ]:
# Ver la distribución de las clases
locality_counts = df['LOCALITY'].value_counts()
print(locality_counts)

# Visualización de la distribución
locality_counts.plot(kind='bar')
plt.title('Distribución de Localidades')
plt.xlabel('Localidad')
plt.ylabel('Número de Registros')
plt.show()


NameError: name 'df' is not defined

La distribución de las clases en la variable "localidad" es bastante equilibrada, ya que cada localidad (Sur, Oriente, Occidente, Norte y Centro) tiene un número similar de registros, con una pequeña diferencia en la localidad Centro, que tiene una instancia menos que las otras localidades. Dado que no se observa un desbalance significativo entre las clases, no es necesario aplicar técnicas de balanceo de clases, como SMOTE o ADASYN, para este conjunto de datos.

## **Modelos Benchmark**

Ya teniendo la certeza de que las clases seencuentran balanceada, se procede entonces a ejecutar los modelos de referencia para el modelo de clasificación, en este caso ejecutaremos K-Nearest Neighbors (KNN), Clasificación de Bayes, Regresión Logística (Regularización L1/L2 - Ridge, Lasso respectivamente), Decision Tree, Random Forest, XGBoost (Lime), y Máquinas de Soporte Vectorial (SVM). Para cada caso analizaremos...

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

# Definir variables predictoras (X) y variable objetivo (y) para clasificación de islas de calor
X = df[[
    'T2M', 'RH2M', 'WS10M', 'PS', 'T2MDEW', 'T2MWET', 'WS50M'
]]

y = df['ICU_Categoria']  # Etiqueta categórica: niveles de isla de calor

# Dividir el dataset en conjunto de entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el modelo KNN
knn = KNeighborsClassifier(n_neighbors=5)

# Entrenar el modelo
knn.fit(X_train, y_train)

# Predecir con el conjunto de prueba
y_pred = knn.predict(X_test)

# Evaluar el modelo: Reporte de clasificación
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

# Matriz de confusión
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

# Curva ROC y AUC
fpr, tpr, thresholds = roc_curve(y_test, knn.predict_proba(X_test), pos_label=1)
roc_auc = roc_auc_score(y_test, y_pred)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.title("Curva ROC para KNN")
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.legend(loc='lower right')
plt.show()

